## Setup

In [1]:
from eesr import interface
from pandas import Timestamp, DataFrame
import pickle
import os
import warnings
import timeit
warnings.filterwarnings("ignore")

from time import process_time, perf_counter

#### Enviroment Info 

In [2]:
%load_ext watermark
%watermark -v -m

Python implementation: CPython
Python version       : 3.10.2
IPython version      : 8.1.1

Compiler    : MSC v.1929 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
CPU cores   : 8
Architecture: 64bit



## Timing Start

In [3]:
process_start = process_time()
perf_start = perf_counter()

In [4]:
# Globals
dc_trace = "Input\opendc_out.csv"
key_path = "Input\entsoe_token.txt"
results_pre_path = "Results"


In [5]:
def run_experiment(name, start: Timestamp, country="NL", tz="Europe/Amsterdam", caching=True, green_ratio=None, subname=None, make_out=True, PUE=1.59):
    if subname is None:
        path = os.path.join(results_pre_path, name)
    else:
        path = os.path.join(results_pre_path, name, subname)
    exists = os.path.exists(path)
    if not exists:
        os.makedirs(path)
    
    def store_df(df: DataFrame):
        pickle_path = os.path.join(path, "df.pkl")
        df.to_pickle(pickle_path)

    results = os.path.join(path, "results.json")
    df = interface.opendc_grid_analysis(dc_trace, key_path, start, country, results, tz, caching, green_ratio, PUE=PUE)

    if make_out:
        store_df(df)

        report_path = os.path.join(path, "report.html")
        report = interface.generate_compact_profile(results, "sus_prof", report_path, True)
        interface.to_image(report)

# Main Experiments

## 1 Caching Performance

#### Timing no cache

In [6]:
%%timeit -n 1 -r 4
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("caching", start, caching=False, make_out=False)

#### Timing cache

In [7]:
%%timeit -n 1 -r 4
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("caching", start, caching=True, make_out=False)

#### Bandwidth savings (with wireshark)

In [8]:
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("caching", start, caching=False, make_out=False)

## 2 Baseline: Assume Proportional to Grid Production

In [9]:
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("baseline", start)

Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'


## 3 Optimistic: Assume 88% Green

In [10]:
start = Timestamp(year=2020, month=9, day=4, hour=15, tz="Europe/Amsterdam")
run_experiment("optimistic", start, green_ratio=0.88)

Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'


## 4 Seasonal Comparison

In [11]:
for i in range(0, 12, 3):
    month = 1 + i
    start = Timestamp(year=2020, month=month, day=1, hour=15, tz="Europe/Amsterdam")
    run_experiment("season", start, subname=str(month))

Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
Only one level for prod fetch
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElement

## 5 Yearly Comparison

In [12]:
pue = {
    2016 : 1.6,
    2017 : 1.59,
    2018: 1.58,
    2019: 1.67,
    2020: 1.59
}

for i in range(5):
    year = 2016 + i
    start = Timestamp(year=year, month=9, day=4, hour=15, tz="Europe/Amsterdam")
    run_experiment("yearly", start, subname=str(year), PUE=pue[year])

Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementB

## 6 Different Countries

In [13]:
countries = {
    "FR" : "Europe/Paris",
    "GB" : "Europe/London",
    "SE_3" : "Europe/Stockholm",
    "BE" : "Europe/Brussels",
    "DE_LU" : "Europe/Berlin"
}

for country in countries.keys():
    start = Timestamp(year=2020, month=9, day=4, hour=15, tz=countries[country])
    run_experiment("countries", start, country, countries[country], subname=country)

Only one level for prod fetch
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
Only one level for prod fetch
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' has no attribute '_getElementById'
fail type object 'Text' h

## Timing End

In [14]:
perf_elapsed_time = perf_counter() - perf_start
process_elapsed_time = process_time() - process_start

write_path = os.path.join(results_pre_path, 'experiment_timing.txt')
with open(write_path, 'w') as f:
    f.write(f"System time: {perf_elapsed_time} \n")
    f.write(f"Process time: {process_elapsed_time}")